# Pinecone Configuration Choices on Downstream App Performance
There are a few important configuration choices to keep in mind when constructing a vector store, e.g. distance metric. In this example, we explore the downstream impact of these configuration choices on response quality, cost & latency of the downstream LLM app.

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["HUGGINGFACE_API_KEY"] = "..."
os.environ["PINECONE_API_KEY"] = "..."
os.environ["PINECONE_ENVIRONMENT"] = "..."

In [2]:
'''!pip install -qU \
  langchain==0.0.162 \
  openai==0.27.7 \
  tiktoken==0.4.0 \
  "pinecone-client[grpc]"==2.2.1 \
  pinecone_datasets=='0.5.0rc10'''

'!pip install -qU   langchain==0.0.162   openai==0.27.7   tiktoken==0.4.0   "pinecone-client[grpc]"==2.2.1   pinecone_datasets==\'0.5.0rc10'

---

🚨 _Note: the above `pip install` is formatted for Jupyter notebooks. If running elsewhere you may need to drop the `!`._

---

## Building the Knowledge Base

We will download a pre-embedding dataset from pinecone-datasets. Allowing us to skip the embedding and preprocessing steps, if you'd rather work through those steps you can find the full notebook here.

In [3]:
import pinecone_datasets

dataset = pinecone_datasets.load_dataset('wikipedia-simple-text-embedding-ada-002-100K')
dataset.head()

,id,values,sparse_values,metadata,blob
0,1-0,"[-0.011254455894231796, -0.01698738895356655, ...",None,None,"{'chunk': 0, 'source': 'https://simple.wikiped..."
1,1-1,"[-0.0015197008615359664, -0.007858820259571075...",None,None,"{'chunk': 1, 'source': 'https://simple.wikiped..."
2,1-2,"[-0.009930099360644817, -0.012211072258651257,...",None,None,"{'chunk': 2, 'source': 'https://simple.wikiped..."
3,1-3,"[-0.011600767262279987, -0.012608098797500134,...",None,None,"{'chunk': 3, 'source': 'https://simple.wikiped..."
4,1-4,"[-0.026462381705641747, -0.016362832859158516,...",None,None,"{'chunk': 4, 'source': 'https://simple.wikiped..."


We'll format the dataset ready for upsert and reduce what we use to a subset of the full dataset.

In [4]:
# we drop sparse_values as they are not needed for this example
dataset.documents.drop(['metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
# we will use rows of the dataset up to index 30_000
dataset.documents.drop(dataset.documents.index[30_000:], inplace=True)
len(dataset)

30000

Now we move on to initializing our Pinecone vector database.

## Vector Database

To create our vector database we first need a [free API key from Pinecone](https://app.pinecone.io). Then we initialize like so:

In [5]:
index_name_v1 = 'langchain-rag-cosine'

In [6]:
import os
import pinecone

# find API key in console at app.pinecone.io
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
# find ENV (cloud region) next to API key in console
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT')

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT
)

if index_name_v1 not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name_v1,
        metric='cosine', # we'll try each distance metric here
        dimension=1536,  # 1536 dim of text-embedding-ada-002
    )

In [7]:
import time

index = pinecone.GRPCIndex(index_name_v1)
# wait a moment for the index to be fully initialized
time.sleep(1)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [8]:
for batch in dataset.iter_documents(batch_size=100):
    index.upsert(batch)

In [9]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.3,
 'namespaces': {'': {'vector_count': 30000}},
 'total_vector_count': 30000}

## Creating a Vector Store and Querying

Now that we've build our index we can switch over to LangChain. We need to initialize a LangChain vector store using the same index we just built. For this we will also need a LangChain embedding object, which we initialize like so:

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings

# get openai api key from platform.openai.com
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

Now initialize the vector store:

In [11]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name_v1)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

## Retrieval Augmented Generation (RAG)

In RAG we take the query as a question that is to be answered by a LLM, but the LLM must answer the question based on the information it is seeing being returned from the `vectorstore`.

To do this we initialize a `RetrievalQA` object like so:

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# completion llm
llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

# Imports main tools for eval
from trulens_eval import TruChain, Feedback, Tru, feedback, Select
import numpy as np
tru = Tru()

# OpenAI as feedback provider
openai = feedback.OpenAI()

# Question/answer relevance between overall question and answer.
qa_relevance = Feedback(openai.relevance).on_input_output()
# By default this will evaluate feedback on main app input and main app output.

# Question/statement relevance between question and each context chunk.
qs_relevance = Feedback(openai.qs_relevance).on_input().on(
    Select.Record.app.combine_documents_chain._call.args.inputs.input_documents[:].page_content
).aggregate(np.mean)

# wrap with TruLens
truchain = TruChain(qa,
    app_id='Chain1_WikipediaQA',
    feedbacks=[qa_relevance, qs_relevance])

✅ In relevance, input prompt will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to *.__record__.main_output or `Select.RecordOutput` .
✅ In qs_relevance, input question will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to *.__record__.app.combine_documents_chain._call.args.inputs.input_documents[:].page_content .
✅ app Chain1_WikipediaQA -> default.sqlite
✅ feedback def. feedback_definition_hash_236b4240a8471a377180e957e378967b -> default.sqlite
✅ feedback def. feedback_definition_hash_71b0de0beebe658aa4b61eba0751f9ab -> default.sqlite


In [46]:
truchain("Name some famous dental floss brands?")
truchain("Which year did Cincinatti become the Capital of Ohio?")
truchain("Which year was Hawaii's state song written?")
truchain("How many countries are there in the world?")
truchain("How many total major trophies has manchester united won?")

✅ record record_hash_a4043cadab1c4aab12c59be9050cd29a from Chain1_WikipediaQA -> default.sqlite
✅ feedback feedback_result_hash_45e5cbbc359d0c4a5507cf4b6624504a on record_hash_a4043cadab1c4aab12c59be9050cd29a -> default.sqlite
✅ feedback feedback_result_hash_440215c7c8d55ae42e0cc0031f6666e9 on record_hash_a4043cadab1c4aab12c59be9050cd29a -> default.sqlite
✅ record record_hash_31b612454566be19b5280d9387f46a7a from Chain1_WikipediaQA -> default.sqlite
✅ record record_hash_3b6b2ea42ad76437cd67b36bc763cb74 from Chain1_WikipediaQA -> default.sqlite
✅ feedback feedback_result_hash_fcc7ed29fa4147a62dfba813109f639a on record_hash_31b612454566be19b5280d9387f46a7a -> default.sqlite
✅ feedback feedback_result_hash_eb88c54cf087b9dced5df1b43db6c7b0 on record_hash_31b612454566be19b5280d9387f46a7a -> default.sqlite
✅ feedback feedback_result_hash_91ff9697f88c79c61947eab204951c71 on record_hash_3b6b2ea42ad76437cd67b36bc763cb74 -> default.sqlite
✅ feedback feedback_result_hash_b735a28ea10437a4d434e071d

{'query': 'How many total major trophies has manchester united won?',
 'result': 'Manchester United has won a total of 66 major trophies.'}

✅ record record_hash_54d474443ee02306ebc6051eb23567a0 from Chain1_WikipediaQA -> default.sqlite
✅ feedback feedback_result_hash_15ad2758d6a16a14c729613e39c2618d on record_hash_3ceb607d84a95f3a58327e1fc914154d -> default.sqlite
✅ feedback feedback_result_hash_4011ab9ee714ee73db6b515150deb253 on record_hash_3ceb607d84a95f3a58327e1fc914154d -> default.sqlite
✅ feedback feedback_result_hash_767b3120bfc08c44ce18f4f88a12815f on record_hash_54d474443ee02306ebc6051eb23567a0 -> default.sqlite
✅ feedback feedback_result_hash_b76df9cc2a6613115eb9f0baa0db57be on record_hash_54d474443ee02306ebc6051eb23567a0 -> default.sqlite


Open the TruLens Dashboard to view tracking and evaluations.

In [14]:
tru.run_dashboard()

✅ record record_hash_e1475327c0a62466222db838cb977700 from Chain1_WikipediaQA -> default.sqlite
Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.4.23:8505 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [17]:
pinecone.delete_index(index_name_v1)
time.sleep(30) # sleep for 30 seconds after deleting the index before creating a new one

In [18]:
index_name_v2 = 'langchain-rag-euclidean'
pinecone.create_index(
        name=index_name_v2,
        metric='euclidean',
        dimension=1536,  # 1536 dim of text-embedding-ada-002
    )

In [21]:
index = pinecone.GRPCIndex(index_name_v2)
# wait a moment for the index to be fully initialized
time.sleep(1)

In [24]:
# qa still exists, and will now use our updated vector store

# recreate qa from vector store
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

# wrap with TruLens
truchain = TruChain(qa,
    app_id='Chain2_WikipediaQA',
    feedbacks=[qa_relevance, qs_relevance])

✅ app Chain2_WikipediaQA -> default.sqlite
✅ feedback def. feedback_definition_hash_6376f76fb6b5ab13f1261d942e8d6729 -> default.sqlite
✅ feedback def. feedback_definition_hash_76c2c7a07a4a8578fe78fdbd173b3288 -> default.sqlite


In [25]:
truchain("Name some famous dental floss brands?")
truchain("Which year did Cincinatti become the Capital of Ohio?")
truchain("Which year was Hawaii's state song written?")
truchain("How many countries are there in the world?")
truchain("How many total major trophies has manchester united won?")

✅ record record_hash_175f73a666abafaabba58a1a0b0a121a from Chain2_WikipediaQA -> default.sqlite
✅ record record_hash_d6ed647224bf101c6f4e265266c55a9c from Chain2_WikipediaQA -> default.sqlite
✅ feedback feedback_result_hash_54b88250207666e46ceb0fc43aaf59d4 on record_hash_175f73a666abafaabba58a1a0b0a121a -> default.sqlite
✅ feedback feedback_result_hash_b5966e85adf5cbbba70018fca5b4f00c on record_hash_175f73a666abafaabba58a1a0b0a121a -> default.sqlite
✅ record record_hash_c9051de489bee46561b9dc2786211414 from Chain2_WikipediaQA -> default.sqlite
✅ feedback feedback_result_hash_f682ba43e0054dafd84f546a90b612c9 on record_hash_c9051de489bee46561b9dc2786211414 -> default.sqlite
✅ feedback feedback_result_hash_ac8915a34dd92474893581f64c693a68 on record_hash_c9051de489bee46561b9dc2786211414 -> default.sqlite
✅ record record_hash_3961d1db52a3db2d762b8ee1cdb4131c from Chain2_WikipediaQA -> default.sqlite


{'query': 'How many total major trophies has manchester united won?',
 'result': 'Manchester United has won a total of 66 major trophies.'}

✅ record record_hash_1e3663ba5074876b91386082ce0a9dc1 from Chain2_WikipediaQA -> default.sqlite
✅ feedback feedback_result_hash_3035174c65658e70bb444482ce7b7553 on record_hash_3961d1db52a3db2d762b8ee1cdb4131c -> default.sqlite
✅ feedback feedback_result_hash_6f19ade4a510f1eb47c1602202ff7b6e on record_hash_3961d1db52a3db2d762b8ee1cdb4131c -> default.sqlite
✅ feedback feedback_result_hash_75c9f85ab2e694000596bf10d6df4899 on record_hash_1e3663ba5074876b91386082ce0a9dc1 -> default.sqlite
✅ feedback feedback_result_hash_58252ee6bb8b914a6fff8ca37e9f4af8 on record_hash_1e3663ba5074876b91386082ce0a9dc1 -> default.sqlite


openai request failed <class 'openai.error.ServiceUnavailableError'>=The server is overloaded or not ready yet.. Retries=3.


✅ feedback feedback_result_hash_333828424de0c468095e5c2eea55be55 on record_hash_d6ed647224bf101c6f4e265266c55a9c -> default.sqlite
✅ feedback feedback_result_hash_6f1a58d86712dce5a755a5c86be4235e on record_hash_d6ed647224bf101c6f4e265266c55a9c -> default.sqlite


In [26]:
pinecone.delete_index(index_name_v2)
time.sleep(30) # sleep for 30 seconds after deleting the index before creating a new one

In [27]:
index_name_v3 = 'langchain-rag-dot'
pinecone.create_index(
        name=index_name_v3,
        metric='dotproduct',
        dimension=1536,  # 1536 dim of text-embedding-ada-002
    )

In [28]:
index = pinecone.GRPCIndex(index_name_v3)
# wait a moment for the index to be fully initialized
time.sleep(1)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [29]:
for batch in dataset.iter_documents(batch_size=100):
    index.upsert(batch)

In [30]:
# switch back to normal index for langchain
index = pinecone.Index(index_name_v3)

# update vectorstore with new index
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

# recreate qa from vector store
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

# wrap with TruLens
truchain = TruChain(qa,
    app_id='Chain3_WikipediaQA',
    feedbacks=[qa_relevance, qs_relevance])

✅ app Chain3_WikipediaQA -> default.sqlite
✅ feedback def. feedback_definition_hash_6376f76fb6b5ab13f1261d942e8d6729 -> default.sqlite
✅ feedback def. feedback_definition_hash_76c2c7a07a4a8578fe78fdbd173b3288 -> default.sqlite


In [31]:
truchain("Name some famous dental floss brands?")
truchain("Which year did Cincinatti become the Capital of Ohio?")
truchain("Which year was Hawaii's state song written?")
truchain("How many countries are there in the world?")
truchain("How many total major trophies has manchester united won?")

✅ record record_hash_34315e45987d717ce831a024177f613d from Chain3_WikipediaQA -> default.sqlite
✅ record record_hash_9f7fe3e6d2b6dc616b5e684f21d0685a from Chain3_WikipediaQA -> default.sqlite
✅ feedback feedback_result_hash_964c4e49f12b368c78354afcc18c0210 on record_hash_9f7fe3e6d2b6dc616b5e684f21d0685a -> default.sqlite
✅ feedback feedback_result_hash_4b8d75a89b85c02d3c37078b43fe2e57 on record_hash_9f7fe3e6d2b6dc616b5e684f21d0685a -> default.sqlite
✅ record record_hash_a441624e136ef679130100513619fd2e from Chain3_WikipediaQA -> default.sqlite
✅ feedback feedback_result_hash_f500a1805826a0be3ca0e81a15575d48 on record_hash_34315e45987d717ce831a024177f613d -> default.sqlite
✅ feedback feedback_result_hash_804a2c8f50149cabc2dcb576bd6abd45 on record_hash_34315e45987d717ce831a024177f613d -> default.sqlite
✅ feedback feedback_result_hash_131d76fe5d6c7f8e2349207aae478020 on record_hash_a441624e136ef679130100513619fd2e -> default.sqlite
✅ feedback feedback_result_hash_3d99bde6bf5e4a99c79dd8a89

{'query': 'How many total major trophies has manchester united won?',
 'result': 'Manchester United has won a total of 66 major trophies.'}

✅ record record_hash_a8d363f88b734f4087cbe876dce51560 from Chain3_WikipediaQA -> default.sqlite
✅ feedback feedback_result_hash_d35ed88a7157b5b5b52ef69a65ccb920 on record_hash_b6e663ef30177500b106f812912ab203 -> default.sqlite
✅ feedback feedback_result_hash_069bac33b69e891ac1ca34f7c54848ca on record_hash_b6e663ef30177500b106f812912ab203 -> default.sqlite
✅ feedback feedback_result_hash_53798f9bda9c540cf0d838e03e496293 on record_hash_a8d363f88b734f4087cbe876dce51560 -> default.sqlite
✅ feedback feedback_result_hash_2a2ae30122b62815ed000760bc5c2b81 on record_hash_a8d363f88b734f4087cbe876dce51560 -> default.sqlite


Our LLM app is performing the fastest with dot-product as our distance metric!

However the app is still hallucinating floss brands that are not supported by context. Let's try a less powerful model with the same grounding approach to cut down on hallucination. This should reduce our token usage as well!

In [32]:
# completion llm
from langchain.llms import OpenAI

llm = OpenAI(
    model_name='text-ada-001',
    temperature=0
)

from langchain.chains import RetrievalQAWithSourcesChain
qa_with_sources = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

# wrap with TruLens
truchain = TruChain(qa_with_sources,
    app_id='Chain4_WikipediaQA',
    feedbacks=[qa_relevance, qs_relevance])

✅ app Chain4_WikipediaQA -> default.sqlite
✅ feedback def. feedback_definition_hash_6376f76fb6b5ab13f1261d942e8d6729 -> default.sqlite
✅ feedback def. feedback_definition_hash_76c2c7a07a4a8578fe78fdbd173b3288 -> default.sqlite


In [33]:
truchain("Name some famous dental floss brands?")
truchain("Which year did Cincinatti become the Capital of Ohio?")
truchain("Which year was Hawaii's state song written?")
truchain("How many countries are there in the world?")
truchain("How many total major trophies has manchester united won?")

✅ record record_hash_21704c168e004a6c39cce0d7bbec8d43 from Chain4_WikipediaQA -> default.sqlite
✅ record record_hash_1ab1ff9ca819234634d6a25f5188c313 from Chain4_WikipediaQA -> default.sqlite
✅ feedback feedback_result_hash_87a5c7d0757453c1189342c0958b7a5a on record_hash_21704c168e004a6c39cce0d7bbec8d43 -> default.sqlite
✅ feedback feedback_result_hash_66bab915bd6dd75e86c4b73c88822a5f on record_hash_21704c168e004a6c39cce0d7bbec8d43 -> default.sqlite
✅ record record_hash_ec5faf1ee050b044ca2e91f40144974a from Chain4_WikipediaQA -> default.sqlite
✅ feedback feedback_result_hash_21481b28953f4ba1faad2412cd9b3a5b on record_hash_1ab1ff9ca819234634d6a25f5188c313 -> default.sqlite
✅ feedback feedback_result_hash_5a12fe4b5ad441727ae21a5fb24c34d5 on record_hash_1ab1ff9ca819234634d6a25f5188c313 -> default.sqlite
✅ record record_hash_368b89f59db91b185e0e9b901fc374db from Chain4_WikipediaQA -> default.sqlite


{'query': 'How many total major trophies has manchester united won?',
 'result': ' manchester united has won a total of 20 league titles, 12 FA Cups, and 3 European Cups.'}

✅ record record_hash_9db722772ce5e624ddf4be907c4ce044 from Chain4_WikipediaQA -> default.sqlite
✅ feedback feedback_result_hash_e080abb10a3731012ced4d4977b53607 on record_hash_ec5faf1ee050b044ca2e91f40144974a -> default.sqlite
✅ feedback feedback_result_hash_d107f7fc7c3ffb2dbbbc3e7777b30245 on record_hash_ec5faf1ee050b044ca2e91f40144974a -> default.sqlite
✅ feedback feedback_result_hash_9b31628f2aa8eee8aa500b9d8a9d54a4 on record_hash_368b89f59db91b185e0e9b901fc374db -> default.sqlite
✅ feedback feedback_result_hash_c7826a828f32a03f81c317b3139d8ce0 on record_hash_368b89f59db91b185e0e9b901fc374db -> default.sqlite
✅ feedback feedback_result_hash_397481cc1cdd6e070aa0422ac2997fff on record_hash_9db722772ce5e624ddf4be907c4ce044 -> default.sqlite
✅ feedback feedback_result_hash_d369f4e5edda663c3657e5de782feb42 on record_hash_9db722772ce5e624ddf4be907c4ce044 -> default.sqlite


This fails because the context exceeds what ada can handle! Let's reduce the "top_k", or the number of documents retrieved in order to reduce the context passed into ada.

In [ ]:
# completion llm
from langchain.llms import OpenAI

llm = OpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0
)

qa= RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(top_k = 1)
)


Note: The way the top_k works with RetrievalQA is that the documents are still retrieved by our semantic search and but only the top_k are passed to the LLM. Howevever TruLens captures all of the context chunks that are being retrieved. In order to calculate an accurate QS Relevance metric that matches what's being passed to the LLM, we need to only calculate the relevance of the top context chunk retrieved.

In [40]:

qs_relevance = Feedback(openai.qs_relevance).on_input().on(
    Select.Record.app.combine_documents_chain._call.args.inputs.input_documents[:1].page_content
).aggregate(np.mean)

# wrap with TruLens
truchain = TruChain(qa,
    app_id='Chain5_WikipediaQA',
    feedbacks=[qa_relevance, qs_relevance])

✅ In qs_relevance, input question will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to *.__record__.app.combine_documents_chain._call.args.inputs.input_documents[:1:].page_content .
✅ app Chain5_WikipediaQA_topk=1_attempt2 -> default.sqlite
✅ feedback def. feedback_definition_hash_6376f76fb6b5ab13f1261d942e8d6729 -> default.sqlite
✅ feedback def. feedback_definition_hash_c60d655f0d9dfbb1aa880a4293fbca84 -> default.sqlite


In [41]:
truchain("Name some famous dental floss brands?")
truchain("Which year did Cincinatti become the Capital of Ohio?")
truchain("Which year was Hawaii's state song written?")
truchain("How many countries are there in the world?")
truchain("How many total major trophies has manchester united won?")

✅ record record_hash_22199b2e9a384d3d86f804b442c4c222 from Chain5_WikipediaQA_topk=1_attempt2 -> default.sqlite
✅ feedback feedback_result_hash_b180ba1bab418a5a935157821f9123bb on record_hash_22199b2e9a384d3d86f804b442c4c222 -> default.sqlite
✅ feedback feedback_result_hash_4e5b962327c0279387e28997be837f6b on record_hash_22199b2e9a384d3d86f804b442c4c222 -> default.sqlite
✅ record record_hash_ef7c819bfa92a82877b8f879b14eb0e1 from Chain5_WikipediaQA_topk=1_attempt2 -> default.sqlite
✅ feedback feedback_result_hash_57570a013f182db42017438ac4829c46 on record_hash_ef7c819bfa92a82877b8f879b14eb0e1 -> default.sqlite
✅ feedback feedback_result_hash_47f7a32e6ac4c35ffa21d655ba5625a0 on record_hash_ef7c819bfa92a82877b8f879b14eb0e1 -> default.sqlite
✅ record record_hash_c1623ecae1dd8efdd0911a8c95c9437e from Chain5_WikipediaQA_topk=1_attempt2 -> default.sqlite
✅ feedback feedback_result_hash_d6a74271faa25221e618df22207be8ab on record_hash_c1623ecae1dd8efdd0911a8c95c9437e -> default.sqlite
✅ feedbac

{'query': 'How many total major trophies has manchester united won?',
 'result': 'Manchester United has won a total of 66 major trophies.'}

✅ record record_hash_7570a2e9b01560b49af05ff6c671dddc from Chain5_WikipediaQA_topk=1_attempt2 -> default.sqlite
✅ feedback feedback_result_hash_bdb07a6b84d813650ad30a6d640047aa on record_hash_7570a2e9b01560b49af05ff6c671dddc -> default.sqlite
✅ feedback feedback_result_hash_4dccf6585d344224eca4573aaaf39381 on record_hash_7570a2e9b01560b49af05ff6c671dddc -> default.sqlite


Blazing fast!